# IFRS17 シミュレーション 金利予実差


* 全てを実行するには、**Cell** メニューから**Run All**を選択します。


## このNotebookについて

``ifrs17sim`` モデルを作成し、選択した1つの新契約モデルポイントについて、IFRSベースでのプロジェクションを複数年間行い、
IFRS17に関する諸数値と次のWaterfallチャートを描写します。

- PV Expected Cashflow Rollforward
- CSM Amortizatoin
- Actual Cashflow RollForwrd
- IFRS17 Financial Performance

実績のプロジェクション年度毎に、IFRSベースの負債を計算するために、期待将来キャッシュフローが展開されます。実績と期待の計算前提はデフォルトで全て同一の値(1.5%)が設定されています。


簡便的なモデルで、特に次の点が大きな簡略化・未対応点です。

- キャッシュフローのタイミングは期始か期末です。
- 事業費は全て保険キャッシュフローに含めています。
- 損失要素はモデル化されておらず、CSMが負を取りえるロジックになっています。
- カバー単位は保有保険金額に設定しています。
- 新契約費の償却率は契約始に固定しています。
- TVOGは無いとしています。
- リスク調整は実装されていません。


## 初期設定、モデルの作成

* 参照するモジュールをインポートします。
* ``ifrs17sim``モデルを作成し、``model``変数に代入します。次に作成するときは、``load_saved``に``True``を設定することによりモデル作成の時間を短縮できます。

In [ ]:
%matplotlib notebook

from draw_charts import (draw_waterfall, 
                         get_waterfalldata,
                         draw_actest_pairs)
import ifrs17sim

model = ifrs17sim.build(load_saved=False)

## モデルポイントの選択

* ``policy_id``に選択するモデルポイントのIDを代入します。
* Actualのプロジェクションを、``proj``、Estimatedのプロジェクションを``inner``に代入します。
* ``inner``は、Actualの時点をインデックスとしてもちます。例えば、``inner[0]``は、t=0時点における期待プロジェクション、``inner[1]``はt=1時点における期待プロジェクション、という具合です。

**演習**
* ``policy_id``を変更し、商品や契約属性が異なることにより、後のテーブルやグラフが変化することを確かめる。

In [ ]:
policy_id = 1
proj = model.OuterProj[policy_id]
inner = proj.InnerProj

## 保険キャッシュフローのグラフ

In [ ]:
import pandas as pd

# %% Draw NetCashflows Graph
data = {'NetCashflows': [proj.NetInsurCF[t] for t in range(50)]}
ax = pd.DataFrame(data).plot.line(marker='o', color='r')

vars = ['PremIncome',
        'BenefitSurr',
        'BenefitDeath',
        'ExpsMaint',
        'ExpsCommTotal',
        'ExpsAcq']

df = proj.cells[vars].to_frame(range(15))

df[vars[1:]] = df[vars[1:]].mul(-1)   # Change outflows to negatives
df.plot(kind='bar', stacked=True, ax=ax, title='Insurance Cashflows')

## 金利の変更

* Actual(実績)のプロジェクション`proj`の`DiscRateAdj[t]` (t年度末の実績金利)に値を設定すると、`t`年度末以降の実績金利が設定した値分だけシフトします。
* それに連動して、t年度末以降の期待キャッシュフローの割引率も、シフト後の金利に変更されます。

In [ ]:
proj.DiscRate.formula

In [ ]:
proj.DiscRate.to_frame((t, 0) for t in range(5))

In [ ]:
proj.DiscRateAdj[2] = 0.01  # 第2年度末以降金利が1%上昇
proj.DiscRate.to_frame((t, 0) for t in range(5))

In [ ]:
inner[1].DiscRate.to_frame(0, 1, 2) # 第1年度末の期待キャッシュフローの割引率 

In [ ]:
inner[2].DiscRate.to_frame(0, 1, 2) # 第2年度末の期待キャッシュフローの割引率 

## PV Expected Cashflow Rollforward

* 一つ目のセルは、各時点の期始の予定の将来キャッシュフロー現価と、現価をその期間でアンワインドした各項目のテーブルデータ(`estcf`)です。
* 二番目のセルは、そのテーブルのデータをWaterfallチャートで表示しています。
* 予実差がある場合、期末時点の実績保有契約数が、その期始時点で予定していた期末保有数と異なるので、転がし計算の結果と差異が生じます。

**演習**
* estcf (DataFrameオブジェクト) の各行の合計が、次の行の`PV_FutureCF`と一致するかどうか確かめる。
* 解約率乗数を変更して、残高や増減項目がどう変化するか確かめる。

In [ ]:
estcf = get_waterfalldata(
            proj,
            items=['PV_FutureCF',
                   'EstPremIncome',
                   'EstIntOnCF',
                   'EstAcqCashflow',
                   'EstClaim',
                   'EstExps'],
            length=3,
            reverseitems=['EstPremIncome'])
estcf

In [ ]:
draw_waterfall(estcf, title='PV Expected Cashflows')

## CSM Amortization

各時点のCSM残高と、CSMの増減項目からなるWaterfal chart用のデータと、グラフです。

**演習**

* 解約率乗数を変更して、CSMの増減項目がどう変化するか確かめる。

In [ ]:
csmrf = get_waterfalldata(
            proj, 
            items=['CSM_Unfloored',
                   'IntAccrCSM',
                   'AdjCSM_FlufCF',
                   'TransServices'],
            length=5,
            reverseitems=['TransServices'])
csmrf

In [ ]:
draw_waterfall(csmrf, title='CSM Amortization')

## Actual Cashflow RollForwrd

実績キャッシュフローのWaterfal chartです。累積キャッシュフローは全て現金として保有していると仮定しています。

In [ ]:
actcf = get_waterfalldata(
            proj,
            items=['PremIncome',
                   'IntAccumCF',
                   'ExpsAcqTotal',
                   'BenefitTotal',
                   'ExpsMaintTotal',
                   'ActualNetCF'],
            length=3,
            reverseitems=['ExpsAcqTotal',
                          'BenefitTotal',
                          'ExpsMaintTotal'])
actcf

In [ ]:
draw_waterfall(actcf, stocks=[0, 5], title='Actual Cashflows')

## IFRS17 Financial Performance

IFRSのPL/BSからなるWaterfal chartです。

**演習**

* 期待キャッシュフロー、CSM、実績キャッシュフローとどのような関係にあるか考察する。
* 対象契約や、出力期間を変更して出力する。
* 解約率乗数を変更して、CSMの増減項目がどう変化するか確かめる。

In [ ]:
ifrspl = get_waterfalldata(
            proj,
            items=['InsurRevenue',
                   'InsurServiceExps',
                   'InsurFinIncomeExps',
                   'ProfitBefTax'],
            length=5,
            reverseitems=['InsurServiceExps'])
ifrspl

In [ ]:
draw_waterfall(ifrspl, stocks=[0, 3], title='IFRS17 Profit/Loss')